<a href="https://colab.research.google.com/github/antrautw/TDIChallenge/blob/master/cheap_strike_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pandas_datareader
!pip install yfinance

  Created wheel for yfinance: filename=yfinance-0.1.54-py2.py3-none-any.whl size=22409 sha256=3db1559d24c3703800487bad5fa3e79a387c1f82cf9f101b0391e13b3571c558
  Stored in directory: /root/.cache/pip/wheels/f9/e3/5b/ec24dd2984b12d61e0abf26289746c2436a0e7844f26f2515c
Successfully built yfinance


In [2]:
#Imports, keys here are pandas_datareader allows us to download easily and
#yfinance allows us to get into yahoo
import pandas_datareader.data as data
import yfinance as yf
import pandas as pd

yf.pdr_override()

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
def cheap_opts(symbol, calls_or_puts):
  '''
  This searches all possible expiry dates and finds contracts.
  '''
  import sys, os
  from datetime import date

  #We're going to suppress prints b/c datareader is annoying then restore printing so this helps
  #old_stdout = sys.stdout
  #sys.stdout = open(os.devnull, "w")

  finaldf = pd.DataFrame()
  ticker = yf.Ticker(symbol)
  stock_price = data.get_data_yahoo(symbol, end_date = date.today())['Close'][-1]
  for opt_date in ticker.options:
    opt = ticker.option_chain(opt_date)
    if calls_or_puts == 'puts':
      opt.puts.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.puts)
    else:
      opt.calls.insert(0,'opt_date', opt_date)
      finaldf = finaldf.append(opt.calls)
  return finaldf, stock_price

In [13]:
%%time

calls_or_puts = 'puts'
symbol = 'zm'

results, stock_price = cheap_opts(symbol, calls_or_puts)

[*********************100%***********************]  1 of 1 completed
CPU times: user 314 ms, sys: 10.3 ms, total: 325 ms
Wall time: 1.27 s


In [14]:
returned = results
# Do some final formatting changes
returned = returned.drop(columns = ['contractSymbol', 'lastTradeDate', 'contractSize', 'currency'])
if calls_or_puts == 'puts':
    returned.insert(3, 'dist OTM', stock_price - returned['strike'])
else:
    returned.insert(3, 'dist OTM', returned['strike'] - stock_price)
  
returned.insert(4, '% OTM', returned['dist OTM']/stock_price*100)
returned['value'] = returned['openInterest']*returned['lastPrice']*100

# Do some filtering
returned = returned[returned['% OTM'] < 35]
returned = returned[returned['lastPrice'] < 15]
#returned = returned[returned['volume'] > 10]
returned = returned[returned['opt_date'] > '2020-04-24']
#returned = returned[returned['percentChange'] < -0]

# Sort and display
returned.sort_values(inplace=True, by='lastPrice', ascending=True)
#returned = returned.head(30)
returned.reset_index(drop=True, inplace=True)
final = returned.style.set_precision(3).set_caption('<H1>' + symbol + ' ' + calls_or_puts.upper() + \
                                          '   (Current = $' + str(round(stock_price, 2)) + ')')
final

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
0,2020-05-01,100.000,0.700,50.060,33.360,0.550,0.800,-0.200,-22.222,141.000,719.000,1.327,False,50330.000
1,2020-05-01,99.000,0.720,51.060,34.026,0.500,0.800,-0.080,-10.000,8.000,100.000,1.344,False,7200.000
2,2020-05-01,103.000,0.750,47.060,31.361,0.650,1.000,-0.300,-28.571,7.000,26.000,1.301,False,1950.000
3,2020-05-01,101.000,0.800,49.060,32.694,0.700,0.800,-0.200,-20.000,1.000,0.000,1.329,False,0.000
4,2020-05-01,102.000,0.900,48.060,32.027,0.600,0.950,-0.150,-14.286,3.000,13.000,1.311,False,1170.000
5,2020-05-01,105.000,0.900,45.060,30.028,0.700,1.000,-0.300,-25.000,14.000,104.000,1.254,False,9360.000
6,2020-05-01,106.000,0.950,44.060,29.362,0.800,1.150,-0.280,-22.764,11.000,39.000,1.267,False,3705.000
7,2020-05-01,107.000,1.070,43.060,28.695,0.850,1.250,-0.140,-11.570,1.000,0.000,1.261,False,0.000
8,2020-05-01,104.000,1.150,46.060,30.694,0.700,1.000,0.000,0.000,1.000,0.000,1.282,False,0.000
9,2020-05-01,108.000,1.150,42.060,28.029,0.400,1.150,-0.260,-18.440,7.000,0.000,1.149,False,0.000


In [0]:
# Download the CSV and you're done!
from google.colab import files
filename = 'cheap_' + symbol + '_' + calls_or_puts + '.csv'

returned.to_csv(filename)
# returned.to_csv('unusual_calls_activity_5_15_exp.csv')
files.download(filename)

In [8]:
%%time
top = returned.sort_values(by='lastPrice', ascending=True) #.head(70)
top.sort_values(by='% OTM', inplace=True, ascending=True)
a = top.style.set_caption(calls_or_puts.upper())
display(a)
# top[top['inTheMoney']== True]

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
203,2020-08-21,97.500000,10.150000,-20.400002,-26.459146,22.850000,23.700000,0.000000,0.000000,11.000000,23.000000,0.519414,1.000000,23345.000000
221,2020-05-15,90.000000,14.450000,-12.900002,-16.731520,13.000000,14.250000,-2.030000,-12.317960,13.000000,24.000000,0.500982,1.000000,34680.000000
220,2020-07-17,87.500000,13.920000,-10.400002,-13.488978,12.650000,13.150000,-6.680000,-32.427185,10.000000,695.000000,0.412726,1.000000,967440.000000
216,2020-05-15,87.500000,12.720000,-10.400002,-13.488978,11.250000,11.650000,-1.280000,-9.142856,17.000000,104.000000,0.532231,1.000000,132288.000000
224,2020-08-21,85.000000,14.800000,-7.900002,-10.246435,11.950000,12.600000,0.000000,0.000000,1.000000,1157.000000,0.426458,1.000000,1712360.000000
200,2020-05-15,85.000000,9.700000,-7.900002,-10.246435,9.200000,9.600000,-4.230001,-30.366120,73.000000,454.000000,0.525395,1.000000,440380.000000
198,2020-05-01,85.000000,9.680000,-7.900002,-10.246435,7.950000,9.200000,-3.929999,-28.875822,14.000000,9.000000,0.521489,1.000000,8712.000000
197,2020-05-08,85.000000,9.250000,-7.900002,-10.246435,6.800000,11.500000,-3.450000,-27.165354,2.000000,4.000000,0.532720,1.000000,3700.000000
211,2020-06-19,85.000000,11.700000,-7.900002,-10.246435,10.300000,10.700000,-2.050000,-14.909092,29.000000,3341.000000,0.446173,1.000000,3908970.000000
214,2020-07-17,85.000000,12.220000,-7.900002,-10.246435,10.950000,11.500000,-2.509999,-17.040050,8.000000,1333.000000,0.427008,1.000000,1628926.000000


CPU times: user 161 ms, sys: 4.87 ms, total: 166 ms
Wall time: 171 ms


In [0]:
#top.to_csv('filtered_puts_05_15.csv')
top.to_csv('filtered_'+calls_or_puts+'_05_15.csv')

In [10]:
otm=returned['inTheMoney'] == False
#returned.loc[otm,'Distance OTM']=999-returned.lastPrice
returned.style.set_precision(2)

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
0,2020-05-01,54.00,0.10,23.10,29.96,0.01,0.34,-0.10,-50.00,6.00,48.00,1.04,0.00,480.00
1,2020-05-01,51.00,0.15,26.10,33.85,0.00,0.18,0.00,0.00,3.00,15.00,1.06,0.00,225.00
2,2020-05-01,52.00,0.15,25.10,32.56,0.00,0.30,0.00,0.00,2.00,12.00,1.11,0.00,180.00
3,2020-05-01,55.00,0.15,22.10,28.66,0.07,0.30,-0.15,-50.00,5.00,656.00,1.01,0.00,9840.00
4,2020-05-01,60.00,0.29,17.10,22.18,0.25,0.35,-0.23,-44.23,125.00,399.00,0.87,0.00,11571.00
5,2020-05-08,55.00,0.31,22.10,28.66,0.09,0.51,-0.19,-38.00,2.00,27.00,0.89,0.00,837.00
6,2020-05-01,61.00,0.32,16.10,20.88,0.25,0.48,-0.34,-51.52,14.00,53.00,0.86,0.00,1696.00
7,2020-05-01,57.00,0.34,20.10,26.07,0.00,0.39,0.00,0.00,1.00,47.00,0.92,0.00,1598.00
8,2020-05-01,63.00,0.35,14.10,18.29,0.00,1.30,-0.49,-58.33,4.00,61.00,0.89,0.00,2135.00
9,2020-05-01,56.00,0.38,21.10,27.37,0.00,4.00,0.00,0.00,18.00,51.00,1.78,0.00,1938.00


In [12]:
top[top['inTheMoney']== True]

,opt_date,strike,lastPrice,dist OTM,% OTM,bid,ask,change,percentChange,volume,openInterest,impliedVolatility,inTheMoney,value
203,2020-08-21,97.5,10.15,-20.400002,-26.459146,22.85,23.70,0.000000,0.000000,11.0,23.0,0.519414,1.0,23345.0
221,2020-05-15,90.0,14.45,-12.900002,-16.731520,13.00,14.25,-2.030000,-12.317960,13.0,24.0,0.500982,1.0,34680.0
220,2020-07-17,87.5,13.92,-10.400002,-13.488978,12.65,13.15,-6.680000,-32.427185,10.0,695.0,0.412726,1.0,967440.0
216,2020-05-15,87.5,12.72,-10.400002,-13.488978,11.25,11.65,-1.280000,-9.142856,17.0,104.0,0.532231,1.0,132288.0
224,2020-08-21,85.0,14.80,-7.900002,-10.246435,11.95,12.60,0.000000,0.000000,1.0,1157.0,0.426458,1.0,1712360.0
200,2020-05-15,85.0,9.70,-7.900002,-10.246435,9.20,9.60,-4.230001,-30.366120,73.0,454.0,0.525395,1.0,440380.0
198,2020-05-01,85.0,9.68,-7.900002,-10.246435,7.95,9.20,-3.929999,-28.875822,14.0,9.0,0.521489,1.0,8712.0
197,2020-05-08,85.0,9.25,-7.900002,-10.246435,6.80,11.50,-3.450000,-27.165354,2.0,4.0,0.532720,1.0,3700.0
211,2020-06-19,85.0,11.70,-7.900002,-10.246435,10.30,10.70,-2.050000,-14.909092,29.0,3341.0,0.446173,1.0,3908970.0
214,2020-07-17,85.0,12.22,-7.900002,-10.246435,10.95,11.50,-2.509999,-17.040050,8.0,1333.0,0.427008,1.0,1628926.0
